<img src="_media/logo_stellantis.png" width="300">

<font size="+3"><b><center>Tutorial 01: Read & Write data</center></b></font>

This notebook explains how to read & write data in HDFS or Oracle.

### Imports

In [1]:
%load_ext autoreload
%autoreload 2
import os
import datetime
import pandas as pd
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 100)
from pyspark.sql import functions as F

from app_template.configuration import spark_config
from app_template.utils import system

### Spark session

In [2]:
spark_context, spark_session = spark_config.get_spark(
    app_name="[app00] Test_Read_Write_Data",
    driver_cores=1,
    driver_mem="4g",
    max_executors=8,
    executor_cores=4,
    executor_mem="4g"
)

### HDFS

#### Read

In [3]:
# Data filepath
flow_filepath = "/user/brc05/data/refined/manf001_vehc_prdc_flow/year=2021/month=02/day=05"

In [4]:
# Read & filter data from HDFS
df_flow = (
    spark_session.read.parquet(flow_filepath)
    .filter(
        (F.col("site_code") == "PY")
        & (F.col("genr_door") == "SMON")
    )
    .select("vin", "faml_grp_labl", "pass_date")
    .orderBy("pass_date")
)

In [5]:
df_flow.persist()

DataFrame[vin: string, faml_grp_labl: string, pass_date: timestamp]

In [6]:
df_flow.limit(5).toPandas()

,vin,faml_grp_labl,pass_date
0,VR1UJZKXZMW005254,e-D34,2021-02-05 04:56:51.155
1,VR1UCYHZSMW003839,t-D34,2021-02-05 05:36:02.575
2,VXKUSHNSSMW003288,P2QO-TH,2021-02-05 06:06:23.077
3,VXKUSHNSSMW003348,P2QO-TH,2021-02-05 06:08:21.313
4,VXKUSHNSSMW003350,P2QO-TH,2021-02-05 06:09:44.501


In [7]:
df_flow.count()

196

#### Write

In [8]:
# Output dirpath
output_dirpath = system.get_hdfs_path(os.path.join(os.environ["UNXDATA"], "prod_flow"))
output_dirpath

'/user/u542310/brc14/app00/data/prod_flow'

In [9]:
# Write data to HDFS
df_flow.coalesce(1).write.parquet(output_dirpath, "overwrite")

### Exadata

**Make sure you have created the *application.yml* file in the *conf* directory and filled in the *brcdb* section (see this [section](https://github.psa-cloud.com/brc14/app00#user-specific-settings) of the *README* for more information)**

#### Instantiate OracleDatabase object

In [10]:
from app_template.infra.oracle_database import OracleDatabase

In [11]:
oracle_db = OracleDatabase(dialect="jdbc", spark_session=spark_session)

Note: The default connector is 'jdbc'. It allows you to perform parallelized read/write. You can also choose 'cx_oracle' connector.

#### Read

In [12]:
# SQL query
query = """
(
    SELECT vin, faml_grp_labl, pass_date
    FROM BRC05.MANF001_VEHC_PRDC_FLOW
    WHERE SITE_CODE = 'PY'
    AND PASS_DATE >= to_date('05/02/21', 'dd/mm/yy')
    AND PASS_DATE < to_date('06/02/21', 'dd/mm/yy')
    AND GENR_DOOR = 'SMON'
)
"""

In [13]:
# Read data from Oracle
df_flow = oracle_db.read_df_from_query(query, fetchsize=20000)

In [14]:
df_flow.persist()

DataFrame[VIN: string, FAML_GRP_LABL: string, PASS_DATE: timestamp]

In [15]:
df_flow.limit(5).toPandas()

,VIN,FAML_GRP_LABL,PASS_DATE
0,VXKUSHNEKMW003477,P2QO-TH,2021-02-05 08:01:17.943
1,VR1URHNJNMW005304,t-D34,2021-02-05 08:25:50.448
2,VXKUSHNSSMW003749,P2QO-TH,2021-02-05 10:03:41.672
3,VR1UCYHZSMW003839,t-D34,2021-02-05 05:36:02.575
4,VXKUSHNSSMW003695,P2QO-TH,2021-02-05 07:35:16.423


In [16]:
df_flow.count()

196

Note:
- The *fetchsize* parameter is set by default to 20000 in the OracleDatabase class (default Oracle row fetch size value is 10). It represents the number of rows to load per network call.
- You can also partition the reading by using the parameters *partition_column*, *lower_bound*, *upper_bound* (see doc).

#### Write

In [17]:
# Write data to Oracle
oracle_db.write_df_to_oracle(
    df_flow,
    "app00_prod_flow_tutorial",
    mode="overwrite"
)

NB: *mode* parameter defines how to behave if the table already exists: 'append', 'overwrite', 'error'.